# Assignment: Data Wrangling
## `! git clone https://github.com/DS3001/wrangling`
## Do Q2, and one of Q1 or Q3.

**Q1.** Open the "tidy_data.pdf" document in the repo, which is a paper called Tidy Data by Hadley Wickham.

  1. Read the abstract. What is this paper about?
  - This paper discusses data cleaning, more specifically "data tidying". Data tidying entails cleaning data to adhere to a certain structure, allowing for a small set of cleaning tools, making it easier to develop tools and tidy the data itself.
  2. Read the introduction. What is the "tidy data standard" intended to accomplish?
  - The intention of the "tidy data standard" is to bring consistency to data cleaning, making it easier by removing a lot of the additional costs of "translation" between different tools. 
  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."
  - "Like families, tidy datasets are all alike but every messy dataset is messy in its own way" means that all tidy datasets are clean, following the same structure, but messy datasets are messy in their own way and require differing approaches to clean them. 
  - The sentence, "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general" says that distinguishing what elements of a dataset are observations and what are variables is generally easy, but coming up with a set definition for observation and variable that can be applied across all datasets is difficult. 
  4. Read Section 2.2. How does Wickham define values, variables, and observations?
  - Value: a datapoint, usually either numbers or strings.
  - Variable: Set of values that measure the same underlying attribute across units.
  - Observation: Set of values measured on the same unit across attribute. 
  5. How is "Tidy Data" defined in section 2.3?
  - "Tidy Data" is a standard of mapping the meaning of a dataset to its structure, adhering to 3 principles:
    1. Each variable forms a column.
    2. Each observation forms a row.
    3. Each type of observational unit forms a table
  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?
  - The 5 most common problems with messy datasets are:
    - Column headers are values, not variable names.
    - Multiple variables are stored in one column.
    - Variables are stored in both rows and columns.
    - Multiple types of observational units are stored in the same table.
    - A single observational unit is stored in multiple tables
  
    The data in Table 4 is messy because the column headers are income ranges and should instead be variable values. "Melting" a dataset involves turning columns into rows. 
  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?
  - Table 11 is messy because there is a column for each possible day in the month. Table 12 is clean because there is a row for each possible day and is molten because it turned the columns from table 11 into rows. 
  8. Read Section 6. What is the "chicken-and-egg" problem with focusing on tidy data? What does Wickham hope happens in the future with further work on the subject of data wrangling?
  - The "chicken-and-egg" problem with focusing on tidy data is that tidy data is only as useful as the tools that work with tidy data and tidy tools are only useful with tidy data. Wickham's hope is that the tidy framework is built upon to develop even better tools and methodologies as a final solution to data wrangling. 

**Q2.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

airbnb_df = pd.read_csv('./data/airbnb_hw.csv')
print("Price data type before:", airbnb_df['Price'].dtypes)

airbnb_df['Price'] = pd.to_numeric(airbnb_df['Price'], errors='coerce')
airbnb_df['price_nan'] = airbnb_df['Price'].isnull()
print("Price data type after:", airbnb_df['Price'].dtypes)
print("Missing values after converting to numeric:", sum(airbnb_df['price_nan']))

Price data type before: object
Price data type after: float64
Missing values after converting to numeric: 181


I cleaned the `Price` variable by coverting it to numeric and coercing the errors. This caused 181 missing values. Further examining the data, it appears value that exceed 999 are formatted as a string with commas (e.g. "1,000"). The commas stop the value from being properly converted to numeric, causing a null value.

In [12]:
airbnb_df = pd.read_csv('./data/airbnb_hw.csv')
print("Price data type before:", airbnb_df['Price'].dtypes)

airbnb_df['Price'] = pd.to_numeric(airbnb_df['Price'].replace({',':''}, regex = True), errors='coerce')
airbnb_df['price_nan'] = airbnb_df['Price'].isnull()
print("Price data type after:", airbnb_df['Price'].dtypes)
print("Missing values after converting to numeric:", sum(airbnb_df['price_nan']))

Price data type before: object
Price data type after: int64
Missing values after converting to numeric: 0


To get around this, I use the replace method to replace the occurences of ',' in the `Price` column. This allows the values over 999 to be properly convered to numeric and there are no 0 missing values.

2. Categorical variable: For the `./data/sharks.csv` data covered in the lecture, clean the "Type" variable as well as you can, and explain the choices you make.

In [66]:
sharks_df = pd.read_csv('./data/sharks.csv', low_memory=False)
print("Type data type: ", sharks_df['Type'].dtype)
print("Type value counts before: ", sharks_df['Type'].value_counts())

dummy = sharks_df['Type']
dummy = dummy.replace(['Sea Disaster', 'Boat', 'Boating','Boatomg'], 'Watercraft')
dummy = dummy.replace(['Invalid', 'Questionable', 'Unconfirmed', 'Unverified', 'Under investigation'], np.nan)
sharks_df['Type'] = dummy

print("Type value counts after: ", sharks_df['Type'].value_counts())


Type data type:  object
Type value counts before:  Type
Unprovoked             4716
Provoked                593
Invalid                 552
Sea Disaster            239
Watercraft              142
Boat                    109
Boating                  92
Questionable             10
Unconfirmed               1
Unverified                1
Under investigation       1
Boatomg                   1
Name: count, dtype: int64
Type value counts after:  Type
Unprovoked    4716
Provoked       593
Watercraft     583
Name: count, dtype: int64


Examining the `Type` variable's value counts, there appears to be 4 main categories, Provoked, Unprovoked, Watercraft related, and unknown. I began by replacing all watercraft related occurences with 'Watercraft'. I then removed all of the seemingly unknown values with null. After cleaning, ther are 4 values for `Type`, Unprovoked, Provoked, Watercraft, and nan. 

3. Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.

In [83]:
url = 'http://www.vcsc.virginia.gov/pretrialdataproject/October%202017%20Cohort_Virginia%20Pretrial%20Data%20Project_Deidentified%20FINAL%20Update_10272021.csv'
df = pd.read_csv(url,low_memory=False)

In [70]:
print("Column values: ", df['WhetherDefendantWasReleasedPretrial'].value_counts())

# replace unclear values with np.nan using dummy variable
dummy = df['WhetherDefendantWasReleasedPretrial']
dummy = dummy.replace(9, np.nan)
dummy = df['WhetherDefendantWasReleasedPretrial'] = dummy
print("Column values: ", df['WhetherDefendantWasReleasedPretrial'].value_counts())

Column values:  WhetherDefendantWasReleasedPretrial
1    19154
0     3801
9       31
Name: count, dtype: int64
Column values:  WhetherDefendantWasReleasedPretrial
1.0    19154
0.0     3801
Name: count, dtype: int64


Examing the `WhetherDefendantWasReleasedPretrial`, I noticed 3 values: 0 meaning no, 1 meaning yes, and 9 which doesn't have a clear meaning. Using a dummy variable, I replaced all the 9 values with np.nan. 

4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [84]:
print("Column values: ", df['ImposedSentenceAllChargeInContactEvent'].value_counts())
print("Column values: ", df['SentenceTypeAllChargesAtConvictionInContactEvent'].value_counts())

imposed = df['ImposedSentenceAllChargeInContactEvent']
sentence_type = df['SentenceTypeAllChargesAtConvictionInContactEvent']

imposed = pd.to_numeric(imposed, errors='coerce')
print(pd.crosstab(imposed.isnull(), sentence_type), '\n')

imposed = imposed.mask(sentence_type == 4, 0)
imposed = imposed.mask(sentence_type == 9, np.nan)

print(pd.crosstab(imposed.isnull(), sentence_type), '\n')

del imposed, sentence_type

Column values:  ImposedSentenceAllChargeInContactEvent
                    9053
0                   4953
12                  1404
.985626283367556    1051
6                    809
                    ... 
49.9712525667351       1
57.0349075975359       1
79.9260780287474       1
42.1642710472279       1
1.6570841889117        1
Name: count, Length: 484, dtype: int64
Column values:  SentenceTypeAllChargesAtConvictionInContactEvent
4    8779
0    8720
1    4299
2     914
9     274
Name: count, dtype: int64
SentenceTypeAllChargesAtConvictionInContactEvent     0     1    2     4    9
ImposedSentenceAllChargeInContactEvent                                      
False                                             8720  4299  914     0    0
True                                                 0     0    0  8779  274 

SentenceTypeAllChargesAtConvictionInContactEvent     0     1    2     4    9
ImposedSentenceAllChargeInContactEvent                                      
False                     

Examining the number of null values in `ImposedSentenceAllChargeInContactEvent` after converting to numeric, we see there are over 9,000 null values. Using the crosstab method, we see the majority of these null values come from the sentence type value 4 and 9. Sentence type 4 is where the charges were dropped, so I replaced all imposed sentence values with sentence type 4 with 0 because charges dropped means no sentence. I also replaced type 9 with null. 